In [3]:
#  read test.json
import json

import pandas as pd
import numpy as np

# train_file.json open
with open('train_file.json') as f:
    data = json.load(f)
#  convert json to pandas dataframe
train = pd.DataFrame(data)

# test_file.json open
with open('test_file.json') as f:
    data = json.load(f)

#  convert json to pandas dataframe
val = pd.DataFrame(data)

In [4]:
#  load the model
from tensorflow.keras.models import load_model
model = load_model('model.h5')


df = pd.concat([train, val])

unique_speakers = set()
for i in range(len(df)):
    
    for sp in (df.iloc[i]['speakers']):
        unique_speakers.add(sp)

print(unique_speakers)
speaker_to_ix = {sp: i for i, sp in enumerate(unique_speakers)}
speaker_to_ix['UNK'] = len(speaker_to_ix)

unique_emotions = set()
for j in range(len(df)):
    for sp in df.iloc[j]['emotions']:
        unique_emotions.add(sp)

print(unique_emotions)
emotion_to_ix = {sp: i for i, sp in enumerate(unique_emotions)}

print(speaker_to_ix)
print(emotion_to_ix)





{'Julie', 'Dr. Stryker Ramoray', 'Flight Attendant', 'Dr. Zane', 'Bonnie', 'Joey/Drake', 'Fireman No. 2', 'Nurse #2', 'Terry', 'Bobby', 'Robert', 'The Woman', 'Paul', 'Paolo', 'Dr. Harad', 'Singer', 'Lauren', 'Chandler', 'PBS Volunteer', 'The Dry Cleaner', 'Student', 'Gunther', 'The Grip', 'Elizabeth', 'Young Ethan', 'Ben', 'Janine', 'Tag', 'Dr. Miller', "Maitre d'", 'Receptionist', 'Joey', 'Kristen', "Richard's Date", 'Voice', 'Guy #1', 'Charlie', 'Rachel', 'A Student', 'Raymond', 'Jim', 'Rachel/actress', 'Max', 'Boy in the Cape', 'The Presenter', 'Emily', 'Isabella', 'Gerston', 'Monica and Ross', 'The Guys', 'Jane', 'Customer', 'Katie', 'Jake', 'Mr. Zelner', 'Monica and Rachel', 'Kori', 'Mike', 'Jill', 'Monica and Phoebe', 'The Lurker', 'Mr. Geller', 'Girl', 'Tony', 'Man', 'The Casting Director', 'Ronni', 'Joanna', 'Older Scientist', 'Kathy', 'Barry', 'Duncan', 'A Female Student', 'Phoebe and Rachel', 'Joanne', 'Sophie', 'Annabelle', 'Lydia', 'Emeril', 'Mr. Heckles', 'an', 'Nurse', '

In [9]:
test=pd.read_json('test_file.json')


          episode                                           speakers  \
0  utterance_3421  [Chandler, Joey, Chandler, Joey, Joey, Chandle...   
1   utterance_119  [Elizabeth, Ross, Paul, Ross, Paul, Ross, Paul...   
2   utterance_510  [Ross, Chandler, Ross, Chandler, Ross, Chandle...   
3   utterance_172         [Rachel, Joey, Rachel, Joey, Rachel, Joey]   
4  utterance_3949  [Monica, Monica, Phoebe, Joey, Joey, Joey, Rac...   

                                            emotions  \
0  [anger, neutral, neutral, surprise, anger, dis...   
1  [neutral, joy, neutral, neutral, anger, surpri...   
2  [neutral, neutral, neutral, joy, fear, joy, su...   
3  [surprise, neutral, disgust, neutral, surprise...   
4  [disgust, disgust, anger, sadness, surprise, a...   

                                          utterances  \
0  [Hey! Hold on a minute, hold on a second. Do y...   
1  [This is my father, Paul Stevens. Dad, this is...   
2  [Ok, bye.  Well, Monica's not coming, it's jus...   
3  [I 

In [7]:
# test the model on testidation dataset
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Concatenate, Dropout, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


max_vocab_size = 10000
max_sequence_length = 50
embedding_dim = 50
lstm_dim = 128

test_utterances=test['utterances']
test_triggers=test['triggers']
test_speakers=test['speakers']
test_emotions=test['emotions']

test_utterances_flat = [item for sublist in test_utterances for item in sublist]
test_triggers_flat = [item for sublist in test_triggers for item in sublist]
test_speaker_flat = [item for sublist in test_speakers for item in sublist]
test_emotion_flat = [item for sublist in test_emotions for item in sublist]

test_speaker_flat = [speaker_to_ix.get(sp, speaker_to_ix['UNK']) for sp in test_speaker_flat]
test_emotion_flat = [emotion_to_ix.get(sp, speaker_to_ix['UNK']) for sp in test_emotion_flat]

# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=max_vocab_size)

test_X_seq = pad_sequences(tokenizer.texts_to_sequences(test_utterances_flat), maxlen=max_sequence_length)


test_X_seq = tf.convert_to_tensor(test_X_seq)


test_X_speaker = tf.convert_to_tensor(test_speaker_flat, dtype=tf.int32)
test_X_emotion = tf.convert_to_tensor(test_emotion_flat, dtype=tf.int32)

# Predict triggers and convert to binary
predicted_triggers = model.predict([test_X_seq, test_X_speaker, test_X_emotion])
predicted_triggers = [1 if i > 0.4 else 0 for i in predicted_triggers]
predicted_triggers = np.array(predicted_triggers)



228/228 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step
